In [1]:
import json
import time

from bs4 import BeautifulSoup
import requests
import pandas as pd

from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options

from tqdm import tqdm

In [2]:
base_url = "https://www.trustpilot.com"

In [3]:
def get_soup(url):
    return BeautifulSoup(requests.get(url).content, 'lxml')

In [5]:
#fetching sub-categories urls:

data = {}

soup = get_soup(base_url + '/categories')
for category in soup.findAll('div', {'class': 'subCategory___BRUDy'}):
    name = category.find('h3', {'class': 'subCategoryHeader___36ykD'}).text
    name = name.strip()
    data[name] = {}
    
    sub_categories = category.find('div', {'class': 'subCategoryList___r67Qj'})
    for sub_category in sub_categories.findAll('div', {'class': 'subCategoryItem___3ksKz'}):
        sub_category_name = sub_category.find('a', {'class': 'navigation___2n5P8'}).text 
        sub_category_uri = sub_category.find('a', {'class': 'navigation___2n5P8'})['href'] 
        data[name][sub_category_name] = sub_category_uri

In [6]:
len(data)

22

In [7]:
data

{'Animals & Pets': {'Animal Health': '/categories/animal_health',
  'Animal Parks & Zoo': '/categories/animal_parks_zoo',
  'Cats & Dogs': '/categories/cats_dogs',
  'Horses & Riding': '/categories/horses_riding',
  'Pet Services': '/categories/pet_services',
  'Pet Stores': '/categories/pet_stores'},
 'Beauty & Well-being': {'Cosmetics & Makeup': '/categories/cosmetics_makeup',
  'Hair Care & Styling': '/categories/hair_care_styling',
  'Personal Care': '/categories/personal_care',
  'Salons & Clinics': '/categories/salons_clinics',
  'Tattoos & Piercings': '/categories/tattoos_piercings',
  'Wellness & Spa': '/categories/wellness_spa',
  'Yoga & Meditation': '/categories/yoga_meditation'},
 'Business Services': {'Administration & Services': '/categories/administration_services',
  'Associations & Centers': '/categories/associations_centers',
  'HR & Recruiting': '/categories/hr_recruiting',
  'IT & Communication': '/categories/it_communication',
  'Import & Export': '/categories/impo

In [ ]:
# data = dict((key,value) for key, value in data.items() if key == 'Events & Entertainment')

In [ ]:
data.pop("Events & Entertainment")
data.pop("Home Services")

In [ ]:
data

In [8]:
# fetch company urls referenced in a given subcategory:

def extract_company_urls_form_page():
    a_list = driver.find_elements_by_xpath('(//div[@class="businessUnitCardsContainer___Qhix1"])/a')
    urls = [a.get_attribute('href') for a in a_list]
    dedup_urls = list(set(urls))
    return dedup_urls

In [9]:
# if a pagination exists on the page:

def go_next_page():
    try:
        button = driver.find_element_by_xpath('//a[@class="paginationLinkNormalize___scOgG paginationLinkNext___1LQ14"]')
        return True, button
    except NoSuchElementException:
        return False, None

In [10]:
# initializing Selenium with a headless firefoxdriver:

options = Options()
options.add_argument('-headless')
options.add_argument('-no-sandbox')
options.add_argument('start-maximized')
options.add_argument('disable-infobars')
options.add_argument("--disable-extensions")

# profile = webdriver.FirefoxProfile()
# profile.set_preference("browser.cache.disk.enable", False)
# profile.set_preference("browser.cache.memory.enable", False)
# profile.set_preference("browser.cache.offline.enable", False)
# profile.set_preference("network.http.use-cache", False)

driver = webdriver.Firefox(executable_path=r"/usr/bin/geckodriver", options=options)

timeout = 3

In [ ]:
# scraping

company_urls = {}
count = 1
for category in tqdm(data):
    for sub_category in tqdm(data[category], leave=False):
        company_urls[sub_category] = []
        print("Category is: ", category)
        print("Sub-Category is: ", sub_category)
        url = base_url + data[category][sub_category] + "?numberofreviews=0&status=all&timeperiod=0"
        driver.get(url)
        print("URL is: ", url)
        
#         current_url =  driver.current_url
#         print("Current URL is: ", current_url)
        
#         driver.implicitly_wait(10)
        
#         if data[category][sub_category] != ((current_url.split('.com'))[1].split('?')[0]):
#             print("Change in Url is detected")
#             data[category][sub_category] = ((current_url.split('.com'))[1].split('?')[0])
#             sub_category = ((current_url.split('/categories/'))[1].split('?')[0])
#             print("Now data[category][sub_category] is : ", data[category][sub_category])
            
        try:
            element_present = EC.presence_of_element_located(
                (By.CLASS_NAME, 'internal___1jK0Z wrapper___26yB4'))
            
            WebDriverWait(driver, timeout).until(element_present)
        except:
            pass
    
        next_page = True
        c = 1

        while next_page:
            #driver.implicitly_wait(10)
            extracted_company_urls = extract_company_urls_form_page()
            company_urls[sub_category] += extracted_company_urls
            next_page, button = go_next_page()
            
            if next_page:
                c += 1
                next_url = base_url + data[category][sub_category] + "?numberofreviews=0&status=all&timeperiod=0" + f'&page={c}'
                driver.get(next_url)
                try: 
                    element_present = EC.presence_of_element_located(
                        (By.CLASS_NAME, 'internal___1jK0Z wrapper___26yB4'))
                    # wrapper___2rOTx
                    WebDriverWait(driver, timeout).until(element_present)
                    
                    
                except:
                    pass

  0%|          | 0/6 [00:00<?, ?it/s]

Category is:  Animals & Pets
Sub-Category is:  Animal Health
URL is:  https://www.trustpilot.com/categories/animal_health?numberofreviews=0&status=all&timeperiod=0



 17%|█▋        | 1/6 [00:10<00:50, 10.14s/it]

Category is:  Animals & Pets
Sub-Category is:  Animal Parks & Zoo
URL is:  https://www.trustpilot.com/categories/animal_parks_zoo?numberofreviews=0&status=all&timeperiod=0



 33%|███▎      | 2/6 [00:15<00:34,  8.75s/it]

...

Category is:  Business Services
Sub-Category is:  Print & Graphic Design
URL is:  https://www.trustpilot.com/categories/print_graphic_design?numberofreviews=0&status=all&timeperiod=0



 64%|██████▎   | 7/11 [36:22<18:31, 277.83s/it]

Category is:  Business Services
Sub-Category is:  Research & Development
URL is:  https://www.trustpilot.com/categories/research_development?numberofreviews=0&status=all&timeperiod=0



 73%|███████▎  | 8/11 [37:11<10:27, 209.11s/it]

Category is:  Business Services
Sub-Category is:  Sales & Marketing
URL is:  https://www.trustpilot.com/categories/sales_marketing?numberofreviews=0&status=all&timeperiod=0


In [ ]:
driver.quit()

In [ ]:
with open('./exports/company_urls_en.json', 'w') as f:
    json.dump(company_urls, f)

In [ ]:
consolidated_data = []

for category in data:
    for sub_category in data[category]:
        for url in company_urls[sub_category]:
            consolidated_data.append((category, sub_category, url))

df_consolidated_data = pd.DataFrame(consolidated_data, columns=['category', 'sub_category', 'company_url'])

df_consolidated_data.to_csv('./exports/consolidate_company_urls.csv', index=False)

In [ ]:
df_consolidated_data.head()